In [1]:
from IPython.core.display import display, HTML
import sys
display(HTML("<style>.container { width:95% !important; }</style>"))
print(sys.version)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [2]:
from datetime import date, datetime
import numpy as np
import pandas as pd
import dataframe_image as dfi
# pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_theme()
sns.set_color_codes()
#pd.set_option('display.max_rows', 300)
#pd.set_option('display.max_columns', 20)

# column_names = ['# Seen', 'ALSA', '# Picked', 'ATA', '# GP', 'GP WR', '# OH', 'OH WR', '# GD', 'GD WR', '# GIH', 'GIH WR', '# GND', 'GND WR', 'IWD', 'Color', 'Rarity']

from Utilities import Logger, Fetcher
import WUBRG
from WUBRG import get_color_identity
from game_metadata import SETS, FORMATS
from game_metadata import Card,CardManager, SetMetadata, FormatMetadata
from data_fetching import DataLoader, LoadedData, DataFramer, FramedData, SetManager, CentralManager
from data_fetching.utils import get_next_17lands_update_time, get_prev_17lands_update_time, get_name_slice, get_color_slice, get_date_slice


TRGT_SET = 'DMU'
LOG_LEVEL = Logger.FLG.DEFAULT
LOG_LEVEL

<Flags.DEFAULT: 3>

# Initialization

In [3]:
data_manager = None
set_data = None
print(f'Available sets: {SETS}')
print(f'Target set: {TRGT_SET}')

Available sets: ['DMU', 'SNC', 'NEO', 'VOW', 'MID']
Target set: DMU


In [4]:
print(f"Current Local Time:  {datetime.now()}")
print(f"Last 17Lands Update: {get_prev_17lands_update_time()}")
print(f"Current UTC Time:    {datetime.utcnow()}")
print(f"Next 17Lands Update: {get_next_17lands_update_time()}")

Current Local Time:  2022-08-28 16:17:25.427908
Last 17Lands Update: 2022-08-28 02:00:00
Current UTC Time:    2022-08-28 18:47:25.428872
Next 17Lands Update: 2022-08-29 02:00:00


In [5]:
start = datetime.utcnow()

if set_data is None:
    set_data = SetManager(TRGT_SET)
set_data.check_for_updates()
card_dict = set_data.SET_METADATA.CARD_DICT

end = datetime.utcnow()
print(f"\n --- Data loaded in {end - start}.")

Loading set metadata for: DMU
Fetching data for set: DMU
Fetching card data for set: DMU
Done!

Checking for missing data for DMU PremierDraft...
DMU PremierDraft has no summary data to get!
DMU PremierDraft has no historic data to get!
Finished checking for missing data for DMU PremierDraft.

Checking for missing data for DMU TradDraft...
DMU TradDraft has no summary data to get!
DMU TradDraft has no historic data to get!
Finished checking for missing data for DMU TradDraft.

Checking for missing data for DMU QuickDraft...
DMU QuickDraft has no summary data to get!
DMU QuickDraft has no historic data to get!
Finished checking for missing data for DMU QuickDraft.


 --- Data loaded in 0:00:04.987928.


## Tier List Analysis

In [6]:
tier_to_rank = {
    "A+": 12,
    "A": 11,
    "A-": 10,
    "B+": 9,
    "B": 8,
    "B-": 7,
    "C+": 6,
    "C": 5,
    "C-": 4,
    "D+": 3,
    "D": 2,
    "D-": 1,
    "F": 0,
    "SB": None,
    "TBD": None
}

rank_to_tier = {v: k for k, v in tier_to_rank.items()}

In [7]:
def fetch_raw_data(url):
    if url.startswith("https://www.17lands.com/tier_list/"):
        guid = url.replace("https://www.17lands.com/tier_list/", "")
        url = f"https://www.17lands.com/card_tiers/data/{guid}"
    fetcher = Fetcher()
    raw_data = fetcher.fetch(url)
    return raw_data


def parse_raw_data(raw_data):
    data = dict()
    for card_rating in raw_data:
        data[card_rating['name']] = {
            'card': card_dict[card_rating['name']], 
            'tier': card_rating['tier'], 
            'rank': tier_to_rank[card_rating['tier']], 
            'synergy': card_rating['flags']['synergy'], 
            'buildaround': card_rating['flags']['buildaround']
        }
    return data


def to_data_frame(data, col_name='Rank'):
    ranks = dict()
    for card in data:
        ranks[card] = data[card]['rank']

    return pd.DataFrame.from_dict(ranks, orient="index", columns=[col_name])


def frame_from_url(url, name):
    raw_data = fetch_raw_data(url)
    data = parse_raw_data(raw_data)
    frame = to_data_frame(data)
    frame.index.name = name
    
    return frame

In [33]:
def merge_rankings(frame_list):
    frame = frame_list[0].copy(True)
    names = list()
    
    for indiv in frame_list:
        name = indiv.index.name
        names.append(name)
        frame[name] = indiv['Rank'].astype('Int64')
        
    frame = frame.drop('Rank', axis=1)
    frame['mean'] = round(frame.mean(axis=1), 3)
    frame['max'] = frame.max(axis=1)
    frame['min'] = frame.min(axis=1)
    frame['range'] = frame['max'] - frame['min']
    frame.index.name = ''
    
    # Get the difference of squares distance to figure out most 'controversial' cards.
    dist = pd.DataFrame()
    for name in names:
        dist[name] = abs(frame['mean'] - frame[name])**2
    dist['dist'] = round(dist.mean(axis=1)**0.5, 1)
    frame['dist'] = dist['dist']

    return frame


def supplement_frame(frame, card_dict):
    cast = dict()
    colors = dict()
    rarities = dict()
    cmc = dict()
    
    
    for card_name in card_dict:
        card = card_dict[card_name]
        cast[card_name] = card.CAST_IDENTITY
        colors[card_name] = card.COLOR_IDENTITY
        rarities[card_name] = card.RARITY
        cmc[card_name] = card.CMC
    #TODO: Populate frame with information about Colours, Costs and Rarities.
    
    series = frame.index.to_series()
    frame['To Cast'] = series.map(cast)
    frame['Identity'] = series.map(colors)
    frame['Rarity'] = series.map(rarities)
    frame['CMC'] = series.map(cmc)
    
    cols = list(frame.columns)
    frame = frame[['CMC', 'Rarity', 'Identity', 'To Cast'] + cols[:-4]]
    return frame

In [16]:
from enum import Flag, auto

class ColorSortStyles(Flag):
    exact = auto()
    subset = auto()
    contains = auto()
    superset = contains
    adjascent = auto()
    shares = auto()
    
    
def _parse_to_set(val, var_name):
    t = type(val)
    if t is str:
        val = set(val)
    elif t is list:
        val = set(val)
    elif t is set:
        pass
    else:
        raise TypeError(f"Invalid type for `{var_name}`. Use `str`, `list`, or `set`.")
        
    ret = set()
    for i in val:
        ret.add(i.upper())
    return ret


def rarity_filter(rarities):
    rarities = _parse_to_set(rarities, 'rarities')
    allowed = set('CURM')
    
    if len(rarities - allowed) != 0:
        raise ValueError(f"Set must be composed of subset of {allowed}")
    
    return lambda frame : frame['Rarity'].isin(rarities)


def cmc_filter(cmc, op):
    if type(cmc) is not int:
        raise ValueError("`cmc` must be an int.")
    
    ops = {
        '>' : lambda frame : frame['CMC'] > cmc,
        '<' : lambda frame : frame['CMC'] < cmc,
        '==' : lambda frame : frame['CMC'] == cmc,
        '!=' : lambda frame : frame['CMC'] != cmc,
        '>=' : lambda frame : frame['CMC'] >= cmc,
        '<=' : lambda frame : frame['CMC'] <= cmc
    }
    
    if op not in ops:
        raise ValueError(f"`op` must be one of {ops}")
    
    return ops[op]


def color_filter(colors, style, col_name):
    colors = _parse_to_set(colors, 'colors')
    allowed = set('WUBRG')
    
    if len(colors - allowed) != 0:
        raise ValueError(f"Set must be composed of subset of {allowed}")
        
    colors = get_color_identity(''.join(colors))
    
    # Does the card colour exactly match the colour filter
    def _exact(frame):
        return lambda frame : frame[col_name] == colors
    
    # Is the card colour a non-strict subset of the colour filter
    def _subset(frame):
        return lambda frame : frame[col_name].isin(WUBRG.get_color_subsets(colors))
    
    # Is the card colour a non-strict superset of the colour filter
    def _superset(frame):
        return lambda frame : frame[col_name].isin(WUBRG.get_color_supersets(colors))
    
    # Is the card colour no more than one colour different of the colour filter
    def _adjascent(frame):
        subset = WUBRG.get_color_subsets(col, len(col) - 1, True)
        superset = WUBRG.get_color_supersets(col, len(col) + 1)
        return lambda frame : frame[col_name].isin(subset + superset)
    
    # Does the card colour share any colour with the colour filter
    def _shares(frame):
        shared = set()
        for color in colors:
            shared = shared.union(set(WUBRG.get_color_supersets(color)))
        return lambda frame : frame[col_name].isin(shared)
    
    funcs = {
        ColorSortStyles.exact : _exact,
        ColorSortStyles.subset : _subset,
        ColorSortStyles.contains : _superset,
        ColorSortStyles.superset : _superset,
        ColorSortStyles.adjascent : _adjascent,
        ColorSortStyles.shares : _shares
    }
    
    if style not in funcs:
        raise ValueError(f"`style` must be one of `ColorSortStyles` enums")
    
    return funcs[style]


def display_frame(frame, order='mean', filters=None):
    pd.set_option('display.max_rows', 300)
    sub_frame = frame
    if filters is not None:
        for f in filters:
            sub_frame = sub_frame[f(sub_frame)]
    
    return sub_frame.sort_values(order, ascending=False)

In [11]:
def calc_avg_rank(data):
    # raw_data = fetch_raw_data(url)
    # data = parse_raw_data(raw_data)

    avg_rank = dict()
    colors = {"W", "U", "B", "R", "G", "", "M"}
    rarities = {"C", "U", "R", "M"}

    for c in colors:
        avg_rank[c] = dict()
        for r in rarities:
            avg_rank[c][r] = {"rank": 0, "count": 0}

    for name in data:
        card = data[name]
        color = card['card'].COLOR_IDENTITY
        rarity = card['card'].RARITY

        if not color in colors:
            color = 'M'

        d = avg_rank[color][rarity]
        d["rank"] = d["rank"] + card["rank"]
        d["count"] = d["count"] + 1


    for c in colors:
        for r in rarities:
            if avg_rank[c][r]["count"] == 0:
                avg_rank[c][r]["avg"] = None
            else:
                avg_rank[c][r]["avg"] = round(avg_rank[c][r]["rank"] / avg_rank[c][r]["count"], 1)

    return avg_rank

In [12]:
zac = frame_from_url("https://www.17lands.com/tier_list/03ab10d25d8841f8aef0aa90b30c434b", 'Zac')
klarm = frame_from_url("https://www.17lands.com/tier_list/3078f70b9a0d415ebf9f555439f5aedb", 'Klarm')
phyrre = frame_from_url("https://www.17lands.com/tier_list/a1ae9a695abb4921866f7f4a9a22e21f", 'Phyrre')
nomad = frame_from_url("https://www.17lands.com/tier_list/19ee3198a63b499bb7c25e6ceaae00ba", 'Nomad')
catharsis = frame_from_url("https://www.17lands.com/tier_list/f1b015031fd741268c5228d4d8435938", 'Catharsis')

In [34]:
base_frame = merge_rankings([zac, klarm, phyrre, nomad, catharsis])
frame = supplement_frame(base_frame, card_dict)
base_frame.mean()

C:\Users\Zachary\AppData\Local\Temp/ipykernel_24184/705340479.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  base_frame.mean()


Zac          5.076628
Klarm        5.544402
Phyrre       5.084291
Nomad        5.855469
Catharsis    5.085271
mean         5.302364
max          6.724138
min          3.942529
range        2.781609
dist         0.896377
CMC          2.858238
dtype: float64

In [35]:
display_frame(frame, order='dist', filters=[rarity_filter('C')])

,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,mean,max,min,range,dist
,,,,,,,,,,,,,,
Deathbloom Gardener,3,C,G,G,2,6,6,2,2,3.600,6.0,2.0,4.0,1.920000
Crystal Grotto,0,C,,,0,1,3,4,5,2.600,5.0,0.0,5.0,1.680000
Toxic Abomination,2,C,B,B,1,5,2,1,4,2.600,5.0,1.0,4.0,1.520000
Writhing Necromass,7,C,B,B,1,4,3,6,5,3.800,6.0,1.0,5.0,1.440000
Impulse,2,C,U,U,5,6,3,3,6,4.600,6.0,3.0,3.0,1.280000
Vanquisher's Axe,1,C,,,1,4,2,1,4,2.400,4.0,1.0,3.0,1.280000
Colossal Growth,2,C,RG,G,3,6,5,6,3,4.600,6.0,3.0,3.0,1.280000
Pixie Illusionist,1,C,UG,U,5,5,7,2,4,4.600,7.0,2.0,5.0,1.280000
Hammerhand,1,C,R,R,5,5,3,2,2,3.400,5.0,2.0,3.0,1.280000


In [36]:
display_frame(frame, order='dist', filters=[rarity_filter('U')])

,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,mean,max,min,range,dist
,,,,,,,,,,,,,,
Choking Miasma,3,U,BG,B,2,2,2,9,4,3.8,9.0,2.0,7.0,2.16
Walking Bulwark,1,U,,,2,5,2,1,5,3.0,5.0,1.0,4.0,1.60
"Raff, Weatherlight Stalwart",2,U,WU,WU,5,5,8,9,8,7.0,9.0,5.0,4.0,1.60
Founding the Third Path,2,U,U,U,4,3,0,1,4,2.4,4.0,0.0,4.0,1.52
"Rona, Sheoldred's Faithful",4,U,UB,UB,6,8,5,7,3,5.8,8.0,3.0,5.0,1.44
Elvish Hydromancer,3,U,UG,G,6,4,3,8,5,5.2,8.0,3.0,5.0,1.44
Wingmantle Chaplain,4,U,W,W,4,7,2,2,4,3.8,7.0,2.0,5.0,1.44
"Rulik Mons, Warren Chief",4,U,RG,RG,6,8,5,3,4,5.2,8.0,3.0,5.0,1.44
Monstrous War-Leech,4,U,UB,B,1,3,2,6,4,3.2,6.0,1.0,5.0,1.44


In [37]:
display_frame(frame, order='dist', filters=[rarity_filter('R')])

,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,mean,max,min,range,dist
,,,,,,,,,,,,,,
Temporal Firestorm,5,R,WUR,R,8,10,3,10,4,7.00,10.0,3.0,7.0,2.80
Drag to the Bottom,4,R,B,B,9,7,3,9,5,6.60,9.0,3.0,6.0,2.08
Defiler of Faith,5,R,W,W,12,10,8,10,5,9.00,12.0,5.0,7.0,2.00
Thran Portal,0,R,,,0,4,4,0,4,2.40,4.0,0.0,4.0,1.92
Ertai Resurrected,4,R,UB,UB,8,8,3,9,10,7.60,10.0,3.0,7.0,1.84
Temporary Lockdown,3,R,W,W,0,3,1,<NA>,5,2.25,5.0,0.0,5.0,1.75
Haughty Djinn,3,R,U,U,9,7,9,10,5,8.00,10.0,5.0,5.0,1.60
Rundvelt Hordemaster,2,R,R,R,0,3,1,0,4,1.60,4.0,0.0,4.0,1.52
The Phasing of Zhalfir,4,R,U,U,3,2,2,7,4,3.60,7.0,2.0,5.0,1.52


In [38]:
display_frame(frame, order='dist', filters=[rarity_filter('M')])

,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,mean,max,min,range,dist
,,,,,,,,,,,,,,
Vesuvan Duplimancy,4,M,U,U,1,2,0,8,5,3.2,8.0,0.0,8.0,2.64
"Jodah, the Unifier",5,M,WUBRG,WUBRG,2,0,0,4,6,2.4,6.0,0.0,6.0,2.08
The World Spell,7,M,G,G,6,1,0,0,2,1.8,6.0,0.0,6.0,1.76
Karn's Sylex,3,M,,,3,6,2,6,6,4.6,6.0,2.0,4.0,1.68
Sol'Kanar the Tainted,5,M,UBR,UBR,10,9,5,9,7,8.0,10.0,5.0,5.0,1.60
"Shanna, Purifying Blade",3,M,WUG,WUG,6,9,4,4,7,6.0,9.0,4.0,5.0,1.60
"Zur, Eternal Schemer",3,M,WUB,WUB,4,6,1,1,3,3.0,6.0,1.0,5.0,1.60
"Sheoldred, the Apocalypse",4,M,B,B,8,9,11,11,8,9.4,11.0,8.0,3.0,1.28
Soul of Windgrace,4,M,BRG,BRG,9,8,8,10,5,8.0,10.0,5.0,5.0,1.20
